# Mixed Effects Model

A Mixed Effects Model (MEM) is a statistical model that incorporates both fixed effects and random effects. Fixed effects are the variables of interest that are consistent across all groups, while random effects account for variability across different groups or levels in the data.

#### What are Mixed Effects Models?
Mixed effects models (or multilevel models) combine **fixed effects** (constant factors across all observations) and **random effects** (variation between groups). In agriculture, fixed effects might include environmental factors (e.g., temperature or fertilizer), while random effects capture variation between farms or fields.

- **Fixed Effects**: Parameters that are consistent across all groups (e.g., effects of fertilizer).
- **Random Effects**: Account for variation between groups (e.g., farm-level or field-level differences).

#### **Key Applications in Agriculture**
1. **Crop Yield Modeling**: Mixed models assess how factors like rainfall, temperature, and fertilizers affect crop yields while accounting for variation across different fields.
   - **Example**: Estimating the effect of fertilizers on crop yields across farms with varying soil qualities.

2. **Agricultural Experimentation**: In field trials, these models account for random variation across experimental plots while estimating treatment effects.
   - **Example**: Testing crop varieties across farms and adjusting for local climatic conditions.

3. **Precision Agriculture**: Models analyze localized data from sensors (e.g., soil moisture or pest presence) to optimize yield predictions.
   - **Example**: Using GPS data to understand how moisture variation impacts crop yield within a field.

4. **Soil Quality & Fertility**: Mixed models estimate the effect of soil treatments while accounting for regional soil variations.
   - **Example**: Studying soil pH changes and their impact on plant growth across different farms.

5. **Pest and Disease Management**: They model how environmental factors affect pest presence while considering field-level variations.
   - **Example**: Assessing how local climate influences pest infestations in different regions.

6. **Livestock Management**: Used to study how factors like diet, environment, and farm size influence livestock health, accounting for individual animal differences.
   - **Example**: Evaluating milk production across farms with varying feed types.

7. **Climate Change Impact**: Mixed models assess how changing climate variables (e.g., temperature, rainfall) affect agricultural outcomes across regions.
   - **Example**: Modeling the impact of increased temperatures on crop yields across various soil types.

#### **Advantages of Mixed Effects Models in Agriculture**
- **Handles Hierarchical Data**: Efficiently analyzes clustered data (e.g., farms or fields).
- **Improves Prediction Accuracy**: Accounts for random variations to provide more reliable predictions.
- **Flexibility**: Can handle both continuous and categorical data.
- **Dealing with Correlated Data**: Corrects for correlations within groups (e.g., fields or farms).
- **Better Generalization**: Avoids overfitting, enhancing model applicability to new data.

#### **Conclusion**
Mixed effects models are essential in agricultural research, offering powerful tools to analyze complex, multi-level data. They are widely used in crop yield prediction, soil analysis, pest management, climate impact studies, and more, making them key for precision agriculture and data-driven decision-making.


In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

# Number of data points (1 Lakh)
n = 100000

# Simulating fixed effects for 15 parameters
temperature = np.random.uniform(18, 30, n)  # Temperature in Celsius
humidity = np.random.uniform(40, 80, n)  # Humidity in percentage
soil_ph = np.random.uniform(5.5, 7.5, n)  # Soil pH
fertilizer = np.random.choice([0, 1], size=n)  # 0 = No fertilizer, 1 = Fertilizer applied
rainfall = np.random.uniform(50, 300, n)  # Rainfall (mm)
sunlight_hours = np.random.uniform(4, 12, n)  # Sunlight hours per day
nitrogen = np.random.uniform(10, 50, n)  # Nitrogen content in soil
soil_treatment = np.random.choice([0, 1], size=n)  # 0 = No treatment, 1 = Soil treated
crop_type = np.random.choice(['wheat', 'rice', 'corn'], size=n)  # Crop type
field_size = np.random.uniform(1000, 5000, n)  # Field size (sq.m)
pH_treatment = np.random.choice([0, 1], size=n)  # pH adjustment treatment
weed_presence = np.random.choice([0, 1], size=n)  # 0 = No, 1 = Yes
pest_presence = np.random.choice([0, 1], size=n)  # 0 = No, 1 = Yes
irrigation = np.random.choice([0, 1], size=n)  # 0 = No irrigation, 1 = Irrigated

# Random effects (variation across fields/farms)
farm_id = np.random.choice(range(1, 51), size=n)  # 50 different farms or regions
field_id = np.random.choice(range(1, 101), size=n)  # 100 different fields

# Simulate Crop Yield
yield_base = 50 + 2*temperature - 0.5*humidity + 1.5*soil_ph + 5*fertilizer + 0.2*rainfall + 0.1*sunlight_hours + 0.8*nitrogen
crop_yield = yield_base + np.random.normal(0, 3, n) + farm_id * np.random.normal(0, 1, n)  # Random farm effects

# Simulate Pest Infestation (0 = No Pest, 1 = Pest)
pest_infestation = 0.1*temperature + 0.05*humidity - 0.2*rainfall + 0.1*sunlight_hours + 0.2*irrigation
pest_infestation = np.random.binomial(1, 1/(1+np.exp(-pest_infestation)))

# Simulate Plant Growth for Soil Quality (pH treatment)
growth_base = 40 + 2*soil_ph + 3*soil_treatment + 1.5*nitrogen
plant_growth = growth_base + np.random.normal(0, 2, n) + field_id * np.random.normal(0, 1, n)  # Random field effects

# Create DataFrame
data = pd.DataFrame({
    'temperature': temperature,
    'humidity': humidity,
    'soil_ph': soil_ph,
    'fertilizer': fertilizer,
    'rainfall': rainfall,
    'sunlight_hours': sunlight_hours,
    'nitrogen': nitrogen,
    'soil_treatment': soil_treatment,
    'crop_type': crop_type,
    'field_size': field_size,
    'pH_treatment': pH_treatment,
    'weed_presence': weed_presence,
    'pest_presence': pest_presence,
    'irrigation': irrigation,
    'farm_id': farm_id,
    'field_id': field_id,  # Adding the 'field_id' column explicitly
    'crop_yield': crop_yield,
    'pest_infestation': pest_infestation,
    'plant_growth': plant_growth
})

# Check if the DataFrame is empty after dropping NaNs
print(f"Data shape after cleaning: {data.shape}")
if data.shape[0] == 0:
    raise ValueError("The DataFrame is empty after cleaning. Check for missing or invalid data.")
else:
    print(f"Data is ready for analysis. Number of rows: {data.shape[0]}")

# Check for non-numeric values or NaNs in the numeric columns
print("Checking for non-numeric or missing values in the data...")
for col in data.columns:
    if data[col].dtype == 'object':  # Checking only object types (strings or non-numeric types)
        print(f"Column '{col}' contains non-numeric data.")

# Replace or drop rows with non-numeric values in the numerical columns
data = data.apply(pd.to_numeric, errors='coerce')  # This will convert non-numeric values to NaN
data = data.dropna()  # Dropping rows with NaN values

# Check distribution of 'farm_id' and 'field_id'
print("Farm ID Distribution (post-cleaning):")
print(data['farm_id'].value_counts())

print("Field ID Distribution (post-cleaning):")
print(data['field_id'].value_counts())

# Check for zero variance columns
zero_variance_columns = data.columns[data.nunique() == 1]
print(f"Columns with zero variance: {zero_variance_columns.tolist()}")

# Mixed Effects Model for Crop Yield (Effect of Fertilizer with Random Effects for Farms)
md = sm.MixedLM.from_formula("crop_yield ~ fertilizer + temperature + humidity + soil_ph + rainfall + sunlight_hours + nitrogen", 
                            data, groups=data["farm_id"])
mdf = md.fit()

# Print the interpretation summaries
def generate_interpretation(model, model_name):
    summary = model.summary().tables[1]
    interpretation = f"\n### {model_name} Interpretation:\n"
    for row in summary[1:]:
        param = row[0]
        p_value = float(row[4])
        coeff = float(row[1])
        
        if p_value < 0.05:
            significance = "Significant"
        else:
            significance = "Not Significant"
        
        interpretation += f"\n- Parameter: {param} | Coefficient: {coeff:.2f} | P-Value: {p_value:.4f} | {significance}"
    
    return interpretation

print(generate_interpretation(mdf, "Crop Yield Model"))


Data shape after cleaning: (100000, 19)
Data is ready for analysis. Number of rows: 100000
Checking for non-numeric or missing values in the data...
Column 'crop_type' contains non-numeric data.
Farm ID Distribution (post-cleaning):
Series([], Name: count, dtype: int64)
Field ID Distribution (post-cleaning):
Series([], Name: count, dtype: int64)
Columns with zero variance: []


ValueError: zero-size array to reduction operation maximum which has no identity

In [4]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

# Number of data points (1 Lakh)
n = 100000

# Simulating fixed effects for 15 parameters
temperature = np.random.uniform(18, 30, n)  # Temperature in Celsius
humidity = np.random.uniform(40, 80, n)  # Humidity in percentage
soil_ph = np.random.uniform(5.5, 7.5, n)  # Soil pH
fertilizer = np.random.choice([0, 1], size=n)  # 0 = No fertilizer, 1 = Fertilizer applied
rainfall = np.random.uniform(50, 300, n)  # Rainfall (mm)
sunlight_hours = np.random.uniform(4, 12, n)  # Sunlight hours per day
nitrogen = np.random.uniform(10, 50, n)  # Nitrogen content in soil
soil_treatment = np.random.choice([0, 1], size=n)  # 0 = No treatment, 1 = Soil treated
crop_type = np.random.choice(['wheat', 'rice', 'corn'], size=n)  # Crop type
field_size = np.random.uniform(1000, 5000, n)  # Field size (sq.m)
pH_treatment = np.random.choice([0, 1], size=n)  # pH adjustment treatment
weed_presence = np.random.choice([0, 1], size=n)  # 0 = No, 1 = Yes
pest_presence = np.random.choice([0, 1], size=n)  # 0 = No, 1 = Yes
irrigation = np.random.choice([0, 1], size=n)  # 0 = No irrigation, 1 = Irrigated

# Random effects (variation across fields/farms)
farm_id = np.random.choice(range(1, 51), size=n)  # 50 different farms or regions
field_id = np.random.choice(range(1, 101), size=n)  # 100 different fields

# Simulate Crop Yield
yield_base = 50 + 2*temperature - 0.5*humidity + 1.5*soil_ph + 5*fertilizer + 0.2*rainfall + 0.1*sunlight_hours + 0.8*nitrogen
crop_yield = yield_base + np.random.normal(0, 3, n) + farm_id * np.random.normal(0, 1, n)  # Random farm effects

# Simulate Pest Infestation (0 = No Pest, 1 = Pest)
pest_infestation = 0.1*temperature + 0.05*humidity - 0.2*rainfall + 0.1*sunlight_hours + 0.2*irrigation
pest_infestation = np.random.binomial(1, 1/(1+np.exp(-pest_infestation)))

# Simulate Plant Growth for Soil Quality (pH treatment)
growth_base = 40 + 2*soil_ph + 3*soil_treatment + 1.5*nitrogen
plant_growth = growth_base + np.random.normal(0, 2, n) + farm_id * np.random.normal(0, 1, n)  # Random field effects

# Create DataFrame (Explicitly add 'field_id')
data = pd.DataFrame({
    'temperature': temperature,
    'humidity': humidity,
    'soil_ph': soil_ph,
    'fertilizer': fertilizer,
    'rainfall': rainfall,
    'sunlight_hours': sunlight_hours,
    'nitrogen': nitrogen,
    'soil_treatment': soil_treatment,
    'crop_type': crop_type,
    'field_size': field_size,
    'pH_treatment': pH_treatment,
    'weed_presence': weed_presence,
    'pest_presence': pest_presence,
    'irrigation': irrigation,
    'farm_id': farm_id,
    'field_id': field_id,  # Ensure 'field_id' is added
    'crop_yield': crop_yield,
    'pest_infestation': pest_infestation,
    'plant_growth': plant_growth
})

# Diagnostic: Check if 'field_id' and 'farm_id' are present
print("Columns in DataFrame:", data.columns)

# Mixed Effects Model for Crop Yield (Effect of Fertilizer with Random Effects for Farms)
md = sm.MixedLM.from_formula("crop_yield ~ fertilizer + temperature + humidity + soil_ph + rainfall + sunlight_hours + nitrogen", 
                            data, groups=data["farm_id"])
mdf = md.fit()

# Mixed Effects Model for Pest Infestation (Effect of Weather on Pest Infestation)
md_pest = sm.MixedLM.from_formula("pest_infestation ~ temperature + humidity + rainfall + sunlight_hours + irrigation", 
                                 data, groups=data["field_id"])
mdf_pest = md_pest.fit()

# Mixed Effects Model for Plant Growth (Effect of Soil Treatment on Growth)
md_growth = sm.MixedLM.from_formula("plant_growth ~ soil_treatment + soil_ph + nitrogen", 
                                   data, groups=data["farm_id"])
mdf_growth = md_growth.fit()

# Function to dynamically generate an interpretation summary
def generate_interpretation(model, model_name):
    summary = model.summary().tables[1]
    interpretation = f"\n### {model_name} Interpretation:\n"
    for row in summary[1:]:
        param = row[0]
        p_value = float(row[4])
        coeff = float(row[1])

        if p_value < 0.05:
            significance = "Significant"
        else:
            significance = "Not Significant"

        interpretation += f"\n- Parameter: {param} | Coefficient: {coeff:.2f} | P-Value: {p_value:.4f} | {significance}"

    return interpretation

# Print the interpretations
print(generate_interpretation(mdf, "Crop Yield Model"))
print(generate_interpretation(mdf_pest, "Pest Infestation Model"))
print(generate_interpretation(mdf_growth, "Plant Growth Model"))

# World-Class Visualizations

# Crop Yield Distribution
plt.figure(figsize=(10, 6))
sns.histplot(data['crop_yield'], kde=True, color='skyblue', bins=30)
plt.title("Crop Yield Distribution", fontsize=16)
plt.xlabel("Crop Yield", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.show()

# Pest Infestation vs Temperature
plt.figure(figsize=(10, 6))
sns.boxplot(x='pest_infestation', y='temperature', data=data, palette='coolwarm')
plt.title("Pest Infestation vs Temperature", fontsize=16)
plt.xlabel("Pest Infestation", fontsize=14)
plt.ylabel("Temperature (Celsius)", fontsize=14)
plt.show()

# Crop Yield vs Fertilizer Application
plt.figure(figsize=(10, 6))
sns.boxplot(x='fertilizer', y='crop_yield', data=data, palette='muted')
plt.title("Crop Yield vs Fertilizer Application", fontsize=16)
plt.xlabel("Fertilizer Applied", fontsize=14)
plt.ylabel("Crop Yield", fontsize=14)
plt.show()

# Sunlight Hours vs Crop Yield
plt.figure(figsize=(10, 6))
sns.scatterplot(x='sunlight_hours', y='crop_yield', data=data, color='orange')
plt.title("Sunlight Hours vs Crop Yield", fontsize=16)
plt.xlabel("Sunlight Hours", fontsize=14)
plt.ylabel("Crop Yield", fontsize=14)
plt.show()

# Pest Infestation vs Soil pH
plt.figure(figsize=(10, 6))
sns.boxplot(x='pest_infestation', y='soil_ph', data=data, palette='Set2')
plt.title("Pest Infestation vs Soil pH", fontsize=16)
plt.xlabel("Pest Infestation", fontsize=14)
plt.ylabel("Soil pH", fontsize=14)
plt.show()

# Temperature vs Pest Infestation
plt.figure(figsize=(10, 6))
sns.scatterplot(x='temperature', y='pest_infestation', data=data, color='red')
plt.title("Temperature vs Pest Infestation", fontsize=16)
plt.xlabel("Temperature (Celsius)", fontsize=14)
plt.ylabel("Pest Infestation", fontsize=14)
plt.show()

# Soil Treatment vs Plant Growth
plt.figure(figsize=(10, 6))
sns.boxplot(x='soil_treatment', y='plant_growth', data=data, palette='viridis')
plt.title("Soil Treatment vs Plant Growth", fontsize=16)
plt.xlabel("Soil Treatment", fontsize=14)
plt.ylabel("Plant Growth", fontsize=14)
plt.show()

# Rainfall vs Pest Infestation
plt.figure(figsize=(10, 6))
sns.scatterplot(x='rainfall', y='pest_infestation', data=data, color='green')
plt.title("Rainfall vs Pest Infestation", fontsize=16)
plt.xlabel("Rainfall (mm)", fontsize=14)
plt.ylabel("Pest Infestation", fontsize=14)
plt.show()

# Nitrogen vs Crop Yield
plt.figure(figsize=(10, 6))
sns.scatterplot(x='nitrogen', y='crop_yield', data=data, color='purple')
plt.title("Nitrogen vs Crop Yield", fontsize=16)
plt.xlabel("Nitrogen Content", fontsize=14)
plt.ylabel("Crop Yield", fontsize=14)
plt.show()

# Crop Type Distribution
plt.figure(figsize=(10, 6))
sns.countplot(x='crop_type', data=data, palette='Set1')
plt.title("Crop Type Distribution", fontsize=16)
plt.xlabel("Crop Type", fontsize=14)
plt.ylabel("Count", fontsize=14)
plt.show()

# Crop Yield by Crop Type
plt.figure(figsize=(10, 6))
sns.boxplot(x='crop_type', y='crop_yield', data=data, palette='magma')
plt.title("Crop Yield by Crop Type", fontsize=16)
plt.xlabel("Crop Type", fontsize=14)
plt.ylabel("Crop Yield", fontsize=14)
plt.show()

# Correlation Heatmap
plt.figure(figsize=(12, 8))
corr_matrix = data[['temperature', 'humidity', 'soil_ph', 'fertilizer', 'rainfall', 'sunlight_hours', 'nitrogen', 'crop_yield']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title("Correlation Heatmap", fontsize=16)
plt.show()

# Crop Yield by Farm
plt.figure(figsize=(10, 6))
sns.boxplot(x='farm_id', y='crop_yield', data=data, palette='coolwarm')
plt.title("Crop Yield by Farm", fontsize=16)
plt.xlabel("Farm ID", fontsize=14)
plt.ylabel("Crop Yield", fontsize=14)
plt.show()

# Plant Growth vs Soil pH
plt.figure(figsize=(10, 6))
sns.scatterplot(x='soil_ph', y='plant_growth', data=data, color='cyan')
plt.title("Plant Growth vs Soil pH", fontsize=16)
plt.xlabel("Soil pH", fontsize=14)
plt.ylabel("Plant Growth", fontsize=14)
plt.show()


Columns in DataFrame: Index(['temperature', 'humidity', 'soil_ph', 'fertilizer', 'rainfall',
       'sunlight_hours', 'nitrogen', 'soil_treatment', 'crop_type',
       'field_size', 'pH_treatment', 'weed_presence', 'pest_presence',
       'irrigation', 'farm_id', 'field_id', 'crop_yield', 'pest_infestation',
       'plant_growth'],
      dtype='object')


g:\venv\venv\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
g:\venv\venv\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
g:\venv\venv\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
g:\venv\venv\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
g:\venv\venv\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
g:\venv\venv\lib\site-packages\statsmodels\base\model.py

ValueError: could not convert string to float: '.'

In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

# Number of data points (1 Lakh)
n = 100000

# Simulating fixed effects for 15 parameters
temperature = np.random.uniform(18, 30, n)  # Temperature in Celsius
humidity = np.random.uniform(40, 80, n)  # Humidity in percentage
soil_ph = np.random.uniform(5.5, 7.5, n)  # Soil pH
fertilizer = np.random.choice([0, 1], size=n)  # 0 = No fertilizer, 1 = Fertilizer applied
rainfall = np.random.uniform(50, 300, n)  # Rainfall (mm)
sunlight_hours = np.random.uniform(4, 12, n)  # Sunlight hours per day
nitrogen = np.random.uniform(10, 50, n)  # Nitrogen content in soil
soil_treatment = np.random.choice([0, 1], size=n)  # 0 = No treatment, 1 = Soil treated
crop_type = np.random.choice(['wheat', 'rice', 'corn'], size=n)  # Crop type
field_size = np.random.uniform(1000, 5000, n)  # Field size (sq.m)
pH_treatment = np.random.choice([0, 1], size=n)  # pH adjustment treatment
weed_presence = np.random.choice([0, 1], size=n)  # 0 = No, 1 = Yes
pest_presence = np.random.choice([0, 1], size=n)  # 0 = No, 1 = Yes
irrigation = np.random.choice([0, 1], size=n)  # 0 = No irrigation, 1 = Irrigated

# Random effects (variation across fields/farms)
farm_id = np.random.choice(range(1, 51), size=n)  # 50 different farms or regions
field_id = np.random.choice(range(1, 101), size=n)  # 100 different fields

# Simulate Crop Yield
yield_base = 50 + 2*temperature - 0.5*humidity + 1.5*soil_ph + 5*fertilizer + 0.2*rainfall + 0.1*sunlight_hours + 0.8*nitrogen
crop_yield = yield_base + np.random.normal(0, 3, n) + farm_id * np.random.normal(0, 1, n)  # Random farm effects

# Simulate Pest Infestation (0 = No Pest, 1 = Pest)
pest_infestation = 0.1*temperature + 0.05*humidity - 0.2*rainfall + 0.1*sunlight_hours + 0.2*irrigation
pest_infestation = np.random.binomial(1, 1/(1+np.exp(-pest_infestation)))

# Simulate Plant Growth for Soil Quality (pH treatment)
growth_base = 40 + 2*soil_ph + 3*soil_treatment + 1.5*nitrogen
plant_growth = growth_base + np.random.normal(0, 2, n) + field_id * np.random.normal(0, 1, n)  # Random field effects

# Create DataFrame
data = pd.DataFrame({
    'temperature': temperature,
    'humidity': humidity,
    'soil_ph': soil_ph,
    'fertilizer': fertilizer,
    'rainfall': rainfall,
    'sunlight_hours': sunlight_hours,
    'nitrogen': nitrogen,
    'soil_treatment': soil_treatment,
    'crop_type': crop_type,
    'field_size': field_size,
    'pH_treatment': pH_treatment,
    'weed_presence': weed_presence,
    'pest_presence': pest_presence,
    'irrigation': irrigation,
    'farm_id': farm_id,
    'crop_yield': crop_yield,
    'pest_infestation': pest_infestation,
    'plant_growth': plant_growth
})

# Mixed Effects Model for Crop Yield (Effect of Fertilizer with Random Effects for Farms)
md = sm.MixedLM.from_formula("crop_yield ~ fertilizer + temperature + humidity + soil_ph + rainfall + sunlight_hours + nitrogen", 
                            data, groups=data["farm_id"])
mdf = md.fit()

# Mixed Effects Model for Pest Infestation (Effect of Weather on Pest Infestation)
md_pest = sm.MixedLM.from_formula("pest_infestation ~ temperature + humidity + rainfall + sunlight_hours + irrigation", 
                                 data, groups=data["field_id"])
mdf_pest = md_pest.fit()

# Mixed Effects Model for Plant Growth (Effect of Soil Treatment on Growth)
md_growth = sm.MixedLM.from_formula("plant_growth ~ soil_treatment + soil_ph + nitrogen", 
                                   data, groups=data["field_id"])
mdf_growth = md_growth.fit()

# Function to dynamically generate an interpretation summary
def generate_interpretation(model, model_name):
    summary = model.summary().tables[1]
    interpretation = f"\n### {model_name} Interpretation:\n"
    for row in summary[1:]:
        param = row[0]
        p_value = float(row[4])
        coeff = float(row[1])
        
        if p_value < 0.05:
            significance = "Significant"
        else:
            significance = "Not Significant"
        
        interpretation += f"\n- Parameter: {param} | Coefficient: {coeff:.2f} | P-Value: {p_value:.4f} | {significance}"
    
    return interpretation

# Print the interpretations
print(generate_interpretation(mdf, "Crop Yield Model"))
print(generate_interpretation(mdf_pest, "Pest Infestation Model"))
print(generate_interpretation(mdf_growth, "Plant Growth Model"))

# Visualizations
sns.histplot(data['crop_yield'], kde=True)
plt.title("Crop Yield Distribution")
plt.show()

sns.boxplot(x='pest_infestation', y='temperature', data=data)
plt.title("Pest Infestation vs. Temperature")
plt.show()

sns.boxplot(x='fertilizer', y='crop_yield', data=data)
plt.title("Crop Yield vs. Fertilizer Application")
plt.show()

sns.scatterplot(x='sunlight_hours', y='crop_yield', data=data)
plt.title("Sunlight Hours vs. Crop Yield")
plt.show()

sns.boxplot(x='pest_infestation', y='soil_ph', data=data)
plt.title("Pest Infestation vs. Soil pH")
plt.show()

sns.scatterplot(x='temperature', y='pest_infestation', data=data)
plt.title("Temperature vs. Pest Infestation")
plt.show()

sns.boxplot(x='soil_treatment', y='plant_growth', data=data)
plt.title("Soil Treatment vs. Plant Growth")
plt.show()

sns.scatterplot(x='rainfall', y='pest_infestation', data=data)
plt.title("Rainfall vs. Pest Infestation")
plt.show()

sns.scatterplot(x='nitrogen', y='crop_yield', data=data)
plt.title("Nitrogen vs. Crop Yield")
plt.show()

sns.countplot(x='crop_type', data=data)
plt.title("Crop Type Distribution")
plt.show()

sns.boxplot(x='crop_type', y='crop_yield', data=data)
plt.title("Crop Yield by Crop Type")
plt.show()

corr_matrix = data[['temperature', 'humidity', 'soil_ph', 'fertilizer', 'rainfall', 'sunlight_hours', 'nitrogen', 'crop_yield']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

sns.boxplot(x='farm_id', y='crop_yield', data=data)
plt.title("Crop Yield by Farm")
plt.show()

sns.scatterplot(x='soil_ph', y='plant_growth', data=data)
plt.title("Plant Growth vs. Soil pH")
plt.show()


g:\venv\venv\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
g:\venv\venv\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
g:\venv\venv\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


KeyError: 'field_id'

In [8]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

# Number of data points (1 Lakh)
n = 100000

# Simulating fixed effects for 15 parameters
temperature = np.random.uniform(18, 30, n)  # Temperature in Celsius
humidity = np.random.uniform(40, 80, n)  # Humidity in percentage
soil_ph = np.random.uniform(5.5, 7.5, n)  # Soil pH
fertilizer = np.random.choice([0, 1], size=n)  # 0 = No fertilizer, 1 = Fertilizer applied
rainfall = np.random.uniform(50, 300, n)  # Rainfall (mm)
sunlight_hours = np.random.uniform(4, 12, n)  # Sunlight hours per day
nitrogen = np.random.uniform(10, 50, n)  # Nitrogen content in soil
soil_treatment = np.random.choice([0, 1], size=n)  # 0 = No treatment, 1 = Soil treated
crop_type = np.random.choice(['wheat', 'rice', 'corn'], size=n)  # Crop type
field_size = np.random.uniform(1000, 5000, n)  # Field size (sq.m)
pH_treatment = np.random.choice([0, 1], size=n)  # pH adjustment treatment
weed_presence = np.random.choice([0, 1], size=n)  # 0 = No, 1 = Yes
pest_presence = np.random.choice([0, 1], size=n)  # 0 = No, 1 = Yes
irrigation = np.random.choice([0, 1], size=n)  # 0 = No irrigation, 1 = Irrigated

# Random effects (variation across fields/farms)
farm_id = np.random.choice(range(1, 51), size=n)  # 50 different farms or regions
field_id = np.random.choice(range(1, 101), size=n)  # 100 different fields

# Simulate Crop Yield
yield_base = 50 + 2*temperature - 0.5*humidity + 1.5*soil_ph + 5*fertilizer + 0.2*rainfall + 0.1*sunlight_hours + 0.8*nitrogen
crop_yield = yield_base + np.random.normal(0, 3, n) + farm_id * np.random.normal(0, 1, n)  # Random farm effects

# Simulate Pest Infestation (0 = No Pest, 1 = Pest)
pest_infestation = 0.1*temperature + 0.05*humidity - 0.2*rainfall + 0.1*sunlight_hours + 0.2*irrigation
pest_infestation = np.random.binomial(1, 1/(1+np.exp(-pest_infestation)))

# Simulate Plant Growth for Soil Quality (pH treatment)
growth_base = 40 + 2*soil_ph + 3*soil_treatment + 1.5*nitrogen
plant_growth = growth_base + np.random.normal(0, 2, n) + field_id * np.random.normal(0, 1, n)  # Random field effects

# Create DataFrame
data = pd.DataFrame({
    'temperature': temperature,
    'humidity': humidity,
    'soil_ph': soil_ph,
    'fertilizer': fertilizer,
    'rainfall': rainfall,
    'sunlight_hours': sunlight_hours,
    'nitrogen': nitrogen,
    'soil_treatment': soil_treatment,
    'crop_type': crop_type,
    'field_size': field_size,
    'pH_treatment': pH_treatment,
    'weed_presence': weed_presence,
    'pest_presence': pest_presence,
    'irrigation': irrigation,
    'farm_id': farm_id,
    'field_id': field_id,  # Adding the 'field_id' column explicitly
    'crop_yield': crop_yield,
    'pest_infestation': pest_infestation,
    'plant_growth': plant_growth
})

# Check if the DataFrame is empty after dropping NaNs
if data.empty:
    raise ValueError("The DataFrame is empty after cleaning. Check for missing or invalid data.")
else:
    print("Data is ready for analysis.")

# Check for non-numeric values or NaNs in the numeric columns
print("Checking for non-numeric or missing values in the data...")
for col in data.columns:
    if data[col].dtype == 'object':  # Checking only object types (strings or non-numeric types)
        print(f"Column '{col}' contains non-numeric data.")

# Replace or drop rows with non-numeric values in the numerical columns
data = data.apply(pd.to_numeric, errors='coerce')  # This will convert non-numeric values to NaN
data = data.dropna()  # Dropping rows with NaN values

# Check distribution of 'farm_id' and 'field_id'
print("Farm ID Distribution:")
print(data['farm_id'].value_counts())

print("Field ID Distribution:")
print(data['field_id'].value_counts())

# Check for zero variance columns
zero_variance_columns = data.columns[data.nunique() == 1]
print("Columns with zero variance:", zero_variance_columns)

# Mixed Effects Model for Crop Yield (Effect of Fertilizer with Random Effects for Farms)
md = sm.MixedLM.from_formula("crop_yield ~ fertilizer + temperature + humidity + soil_ph + rainfall + sunlight_hours + nitrogen", 
                            data, groups=data["farm_id"])
mdf = md.fit()

# Mixed Effects Model for Pest Infestation (Effect of Weather on Pest Infestation)
md_pest = sm.MixedLM.from_formula("pest_infestation ~ temperature + humidity + rainfall + sunlight_hours + irrigation", 
                                 data, groups=data["field_id"])
mdf_pest = md_pest.fit()

# Mixed Effects Model for Plant Growth (Effect of Soil Treatment on Growth)
md_growth = sm.MixedLM.from_formula("plant_growth ~ soil_treatment + soil_ph + nitrogen", 
                                   data, groups=data["field_id"])
mdf_growth = md_growth.fit()

# Function to dynamically generate an interpretation summary
def generate_interpretation(model, model_name):
    summary = model.summary().tables[1]
    interpretation = f"\n### {model_name} Interpretation:\n"
    for row in summary[1:]:
        param = row[0]
        p_value = float(row[4])
        coeff = float(row[1])
        
        if p_value < 0.05:
            significance = "Significant"
        else:
            significance = "Not Significant"
        
        interpretation += f"\n- Parameter: {param} | Coefficient: {coeff:.2f} | P-Value: {p_value:.4f} | {significance}"
    
    return interpretation

# Print the interpretations
print(generate_interpretation(mdf, "Crop Yield Model"))
print(generate_interpretation(mdf_pest, "Pest Infestation Model"))
print(generate_interpretation(mdf_growth, "Plant Growth Model"))


Data is ready for analysis.
Checking for non-numeric or missing values in the data...
Column 'crop_type' contains non-numeric data.
Farm ID Distribution:
Series([], Name: count, dtype: int64)
Field ID Distribution:
Series([], Name: count, dtype: int64)
Columns with zero variance: Index([], dtype='object')


ValueError: zero-size array to reduction operation maximum which has no identity